In [1]:
import pandas as pd

from rectools import Columns

import dill
from implicit.nearest_neighbours import TFIDFRecommender
from rectools import Columns
from rectools.dataset import Dataset

import sys
sys.path.append("../")

from rec_sys.models.user_knn import UserKnn
from rec_sys.models.popular import Popular

c:\Users\ODOBESKURS\ITMO\1 term\recsys-itmo\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = "../data"
interactions = pd.read_csv(f'{path}/interactions.csv') # .sample(frac=0.01)
users = pd.read_csv(f'{path}/users.csv')
items = pd.read_csv(f'{path}/items.csv')

In [3]:
interactions.rename(
    columns={
        'last_watch_dt': Columns.Datetime,
        'total_dur': Columns.Weight
    },
    inplace=True,
)

interactions['datetime'] = pd.to_datetime(interactions['datetime'])

In [4]:
dataset = Dataset.construct(
            interactions_df=interactions,
            user_features_df=None,
            item_features_df=None,
        )

Обучим кастомную (чтобы работала с холодными юзерами) популярную модельку, которая будет дополнять предсказания UserKNN

In [5]:
model_popular = Popular(interactions=interactions)
model_popular.fit(dataset)

In [6]:
with open('../models/popular.dill', 'wb') as f:
    dill.dump(model_popular, f)

In [7]:
with open('../models/popular.dill', 'rb') as f:
    model_popular = dill.load(f)

Популярная моделька передается в кнн, чтобы дополнить до 10 рекомендаций

In [8]:
model = TFIDFRecommender()
userknn_model = UserKnn(model=model, N_users=50, interactions=interactions, add_popular=True, model_popular=model_popular)

Обучим UserKNN

In [9]:
userknn_model.fit(interactions)

c:\Users\ODOBESKURS\ITMO\1 term\recsys-itmo\.venv\lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.2199852466583252 seconds
  warnings.warn(
100%|██████████| 962179/962179 [19:50<00:00, 808.25it/s] 


In [10]:
with open('../models/userknn_tfidf_50.dill', 'wb') as f:
    dill.dump(userknn_model, f)

In [11]:
with open('../models/userknn_tfidf_50.dill', 'rb') as f:
    userknn_model = dill.load(f)

In [12]:
print(userknn_model.interactions)

         user_id  item_id   datetime  weight  watched_pct
0         176549     9506 2021-05-11    4250         72.0
1         699317     1659 2021-05-29    8317        100.0
2         656683     7107 2021-05-09      10          0.0
3         864613     7638 2021-07-05   14483        100.0
4         964868     9506 2021-04-30    6725        100.0
...          ...      ...        ...     ...          ...
5476246   648596    12225 2021-08-13      76          0.0
5476247   546862     9673 2021-04-13    2308         49.0
5476248   697262    15297 2021-08-20   18307         63.0
5476249   384202    16197 2021-04-19    6203        100.0
5476250   319709     4436 2021-08-15    3921         45.0

[5476251 rows x 5 columns]


In [13]:
userknn_reco_df = userknn_model.predict(interactions.sample(frac=0.01), N_recs=10)

In [14]:
userknn_reco_df

,user_id,item_id,score,rank
2,1097557,3182,6.137841,1
0,1097557,4151,4.111983,2
1,1097557,15297,3.379502,3
3,1097557,10440,3.029132,4
110,1097513,6674,2.545422,1
...,...,...,...,...
2250207,0,7829,3.915136,6
2250209,0,12192,3.466101,7
2250202,0,9728,2.683715,8
2250203,0,10440,2.390548,9


predict_online отличается тем, что работает на холодных пользователях и дополняет рекомендации до 10 с помощью popular

In [15]:
userknn_model.predict_online([[1097045]])

array([7460, 9550, 10958, 5469, 957, 5287, 13414, 5250, 13325, 2220],
      dtype=object)

In [16]:
userknn_model.predict_online([[864613]])

array([7638, 7189, 4260, 13668, 6006, 7476, 12192, 4151, 15297, 10440],
      dtype=object)

In [17]:
userknn_model.predict_online([[999999999999]])

array([10440., 15297.,  9728., 13865.,  4151.,  3734.,  2657.,  4880.,
         142.,  6809.])